In [7]:
import os

from dotenv import load_dotenv
from openai import OpenAI, AzureOpenAI
import requests


In [2]:
load_dotenv()

True

In [3]:
# client = OpenAI(
#   api_key = os.getenv("OPENAI_API_KEY"),
# )

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

### Empower ChatGPT with Google Search skill.

Custom Search JSON API 
- [Create the Programmable Search Engine](https://programmablesearchengine.google.com/controlpanel/all)
- [Get an API key](https://developers.google.com/custom-search/v1/overview#api_key)
- Custom Search JSON API provides 100 search queries per day for free.

Reference: https://developers.google.com/custom-search/v1/overview

In [4]:
def google_search(query: str) -> list[dict]:
    """ Google search API

        Return a dict of custom search result, see more detail at
        https://developers.google.com/custom-search/v1/reference/rest/v1/Search#result
    """
    url = 'https://www.googleapis.com/customsearch/v1'
    params = {
        'q': query,
        'key': os.getenv("GOOGLE_SEARCH_API_KEY"),
        'cx': os.getenv("GOOGLE_SEARCH_ENGINE_ID"),
    }
    response = requests.get(url, params=params)
    results = response.json().get('items', [])
    return results

In [5]:
system_prompt = "當不確定使用者的問題時，請回答「不知道」。"
user_prompt = "請問 2023 年金馬獎最佳男配角是誰？"

completion = client.chat.completions.create(
  model="gpt-35-turbo-120",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ]
)
response = completion.choices[0].message.content

print(f'Prompt: {user_prompt}')
print(f'Completion: {response}')

if '不知道' in response:
    print('No answer from AI model, using Google search instead ...')

    system_prompt = "以下為已發生的事實：\n\n"
    results = google_search(user_prompt)
    for item in results:
        system_prompt += f"標題：{item['title']}\n摘要：{item['snippet']}\n\n"
    system_prompt += "請依照上述事實回答以下問題。\n"

    completion = client.chat.completions.create(
        model="gpt-35-turbo-120",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    response = completion.choices[0].message.content

    print(f'Completion: {response}')


Prompt: 請問 2023 年金馬獎最佳男配角是誰？
Completion: 不知道。
No answer from AI model, using Google search instead ...
Completion: 根據上述資料，2023年金馬獎最佳男配角得主是陳慕義（出演《老狐狸》）。


In [6]:
print(system_prompt)

以下為已發生的事實：

標題：【金馬60】2023金馬獎得獎名單！
摘要：Nov 25, 2023 ... 金馬60「最佳新演員」得獎者. 金馬得獎金馬60 2023金馬男主角女主角 FB@但願人長久.

標題：【金馬60】2023金馬獎得獎名單！吳慷仁拿下最佳男主角，林品彤是 ...
摘要：Nov 25, 2023 ... 金馬60得獎名單一次看！﻿2023年金馬獎﻿討論度最高的就是最佳男主角阮經天、吳慷仁、許光漢、王柏傑、林柏宏，另外最佳女主角陸小芬、﻿余香凝、﻿ ...

標題：金馬獎最佳男配角- 维基百科，自由的百科全书
摘要：金馬獎最佳男配角是由財團法人中華民國電影事業發展基金會的臺北金馬影展執行 ... 2023年. 官方网站, www.goldenhorse.org.tw. 电视／电台转播. 電視網, 詳見轉播單位.

標題：【2023金馬獎】入圍名單！許光漢林柏宏搶影帝，關於我和鬼爭最佳 ...
摘要：Mar 1, 2024 ... 2023金馬獎入圍名單亮點. 最佳劇情片; 最佳導演; 最佳新導演; 最佳男主角; 最佳女主角; 最佳男配角; 最佳女配角; 最佳新演員. 2023金馬獎完整入圍名單出爐.

標題：金马奖最佳男配角- 维基百科，自由的百科全书
摘要：金马奖最佳男配角是 ... 中华民国（台湾）. 主办单位, 财团法人中华民国电影事业发展基金会 · 台北金马影展执行委员会. 奖励, 金马奖奖座. 首次颁发, 1962年. 最近颁发, 2023 ...

標題：【2023 金馬獎】金馬60 最佳男配角入圍者分析：陳澤耀《富都青年 ...
摘要：Nov 13, 2023 ... 【2023 金馬獎】金馬60「最佳男配角」激烈戰況不輸男主角：男團成員突破偶像框架呼聲高、11 歲小演員超齡表現賺人熱淚！ By Nancy.Chen. 2023年11月13日.

標題：2023金馬獎得獎名單一覽！吳慷仁是最佳男主角、天才少女「林品彤 ...
摘要：Nov 26, 2023 ... 李鴻其《愛是一把槍》 孫杰《大山來了》. 金馬60得獎名單：最佳男配角. 得獎著：陳慕義《老狐狸》. 陳澤耀《富都青年》 黃梓樂《年少日記》 陳慕義 ...

標題：無影無蹤- 【2023金馬獎「最佳男主角、男配角」戰況：男主五帥之爭 ...
摘要：Nov 23, 202